In [18]:
import pandas as pd
import numpy as np

account_file = 'data/account_dim_20250325.csv'
account_df = pd.read_csv(account_file, dtype={'account_id': str})

print(account_df.info())
# account_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18070 entries, 0 to 18069
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   current_account_nbr          18070 non-null  object 
 1   client_id                    18070 non-null  object 
 2   open_date                    18070 non-null  object 
 3   card_activation_date         15223 non-null  object 
 4   card_activation_flag         18070 non-null  object 
 5   ebill_ind                    18052 non-null  object 
 6   date_in_collection           0 non-null      float64
 7   overlimit_type_flag          18070 non-null  int64  
 8   payment_hist_1_12_mths       18070 non-null  object 
 9   payment_hist_13_24_mths      18070 non-null  object 
 10  employee_code                3003 non-null   object 
 11  external_status_reason_code  18070 non-null  int64  
 12  special_finance_charge_ind   0 non-null      float64
 13  pscc_ind        

# Data preprocessing

In [ ]:
def pre_process_accounts(account_df):
    
    # drop duplicate rows
    account_df.drop_duplicates(inplace=True)
    
    # drop useless columns
    useless_column  = ['date_in_collection', # All values are null
                       'special_finance_charge_ind', # All values are null
                       'card_activation_date', # Redundant since we have card_activation_flag
                       'ebill_ind', # Not usefull for computing credit_line increase
                       'overlimit_type_flag', # All values are 0
                       'external_status_reason_code' # Redundant with ext_status_reason_cd_desc
                       ]
    account_df.drop(columns=useless_column, inplace=True, errors='ignore')

    # convert date columns to datetime
    date_columns = ['open_date']
    for col in date_columns:
        account_df[col] = pd.to_datetime(account_df[col], errors='coerce')

    # update the card_activation_flag to 0 or 1. Currently it has the following values: 0, 7, 8 and \\\\\\"". 0 mean activated and the rest are not activated.
    account_df['card_activation_flag'] = account_df['card_activation_flag'].replace({'0': 1, '7': 0, '8': 0, '\\\\\\""': 0})

    # Clean and split payment_history column
    account_df['payment_hist_1_12_mths'] = account_df['payment_hist_1_12_mths'].replace('\\\\\\""', np.nan)
    account_df['payment_hist_13_24_mths'] = account_df['payment_hist_13_24_mths'].replace('\\\\\\"', np.nan)
    # Create empty columns for each month
    for i in range(24):
        account_df[f"Month_{i+1}"] = account_df['payment_hist_1_12_mths'].apply(lambda x: x[i] if pd.notna(x) and i < len(x) else np.nan)
        account_df[f"Month_{i+13}"] = account_df['payment_hist_13_24_mths'].apply(lambda x: x[i] if pd.notna(x) and i < len(x) else np.nan)

    # Update the employee_code column. Currently it has the following values: H, Y and empty/na/null. Y mean employee, H means high spending customer and empty/na/null means normal customer. I want a separate column for high spending customer and employee. The rest are normal customers.
    account_df['high_spending_customer'] = account_df['employee_code'].replace({'H': 1, 'Y': 0, '': 0, np.nan: 0})
    account_df['employee_code'] = account_df['employee_code'].replace({'Y': 1, 'H': 0, '': 0, np.nan: 0})

    

    



    return account_df


